In [7]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC, SVR
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, roc_curve, \
RocCurveDisplay, roc_auc_score, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import log_loss
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [8]:
hr = pd.read_csv('../datasets/HR_comma_sep.csv')
hr.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [15]:
ohe = OneHotEncoder(drop = 'first', sparse_output = False)
col_trnf = ColumnTransformer([('OHE', ohe,make_column_selector(dtype_include=object) )],
                             remainder='passthrough',
                             verbose_feature_names_out=False)

col_trnf = col_trnf.set_output(transform = 'pandas')


In [16]:
x,y = hr.drop('left', axis =1), hr['left']
x = col_trnf.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=25, test_size=0.3, stratify=y)

# KNN + Scaling

In [17]:
scaler = StandardScaler()
knn = KNeighborsClassifier(n_neighbors=5)

pipe = Pipeline([('Scaling', scaler), ('KNN', knn)])
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9402089353189598


# SVM + Scaling

In [19]:
scaler = StandardScaler()
svm = SVC(kernel='rbf')

pipe = Pipeline([('Scaling', scaler), ('SVM', svm)])
pipe.fit(x_train, y_train)
y_pred = pipe.predict(x_test)

print(accuracy_score(y_test, y_pred))

0.9524338741942654


using accuracy

In [23]:
from sklearn.ensemble import VotingClassifier

scaler = StandardScaler()

knn = KNeighborsClassifier()
pipe_knn  = Pipeline([('Scaling', scaler), ('SVM', knn)])

svm = SVC(kernel='rbf', random_state=25, probability=True)
pipe_svm = Pipeline([('Scaling', scaler), ('SVM', svm)])

dtc1 = DecisionTreeClassifier(random_state=25, max_depth=4)
voting = VotingClassifier(estimators = [('Tree1', dtc1),('SVM', pipe_svm),('KNN', pipe_knn)],
                          voting = 'soft')

voting.fit(x_train, y_train)
y_pred = voting.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.9666592576128028


using log_loss

In [27]:
scaler = StandardScaler()

knn1 = KNeighborsClassifier(n_neighbors=5)
knn2 = KNeighborsClassifier(n_neighbors=3)
pipe_knn1  = Pipeline([('Scaling', scaler), ('SVM', knn1)])
pipe_knn2  = Pipeline([('Scaling', scaler), ('SVM', knn2)])

svm = SVC(kernel='rbf', random_state=25, probability=True)
pipe_svm = Pipeline([('Scaling', scaler), ('SVM', svm)])

dtc1 = DecisionTreeClassifier(random_state=25, max_depth=7)
dtc2 = DecisionTreeClassifier(random_state=25, max_depth=4)
voting = VotingClassifier(estimators = [('Tree1', dtc1),('SVM', pipe_svm),('KNN1', pipe_knn1),('KNN2', pipe_knn2)],
                          voting = 'soft')

voting.fit(x_train, y_train)
y_pred_proba = voting.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))

0.09420120353483613
